In [39]:
import tensorflow as tf
from tensorflow.contrib.learn.python.learn.datasets.mnist import read_data_sets
import math
graph = tf.Graph()
session = tf.InteractiveSession(graph=graph)

mnist = read_data_sets("data", one_hot=True, reshape=False)

"parameter"
epoch = 15
batch_size = 100
keep_prob = tf.placeholder(tf.float32)
lr = tf.placeholder(tf.float32)

X = tf.placeholder(tf.float32, shape=[None, 28, 28, 1])
X2 = tf.reshape(X, [-1, 784])
Y = tf.placeholder(tf.float32, shape=[None, 10])

W1 = tf.Variable(tf.random_normal([784, 512], stddev=0.1))
b1 = tf.Variable(tf.random_normal([512], stddev=-0.1))
L1 = tf.nn.relu(tf.matmul(X2, W1)+b1)
L1 = tf.nn.dropout(L1, keep_prob=keep_prob)

W2 = tf.Variable(tf.random_normal([512, 256], stddev=0.1))
b2 = tf.Variable(tf.random_normal([256], stddev=-0.1))
L2 = tf.nn.relu(tf.matmul(L1, W2)+b2)
L2 = tf.nn.dropout(L2, keep_prob=keep_prob)

W3 = tf.Variable(tf.random_normal([256, 128], stddev=0.1))
b3 = tf.Variable(tf.random_normal([128], stddev=-0.1))
L3 = tf.nn.relu(tf.matmul(L2, W3)+b3)
L3 = tf.nn.dropout(L3, keep_prob=keep_prob)

W4 = tf.Variable(tf.random_normal([128, 64], stddev=0.1))
b4 = tf.Variable(tf.random_normal([64], stddev=-0.1))
L4 = tf.nn.relu(tf.matmul(L3, W4)+b4)
L4 = tf.nn.dropout(L4, keep_prob=keep_prob)

W5 = tf.Variable(tf.random_normal([64, 10], stddev=0.1))
b5 = tf.Variable(tf.random_normal([10], stddev=-0.1))
logits = tf.matmul(L4, W5)+b5
H = tf.nn.softmax(logits)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=Y))*100
train = tf.train.AdamOptimizer(lr).minimize(cost)

is_correct = tf.equal(tf.argmax(H, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

sess = tf.Session()
sess.run(tf.global_variables_initializer())


c = 0
avg_cost = 0
for e in range(epoch):
    total_batch = int(mnist.train.num_examples/batch_size)
    global avg_cost
    global c
    print(e+1)
    for i in range(total_batch):
        
        max_learning_rate = 0.003
        min_learning_rate = 0.0001
        decay_speed = 2000.0
        learning_rate = min_learning_rate + (max_learning_rate - min_learning_rate) *math.exp(-i/decay_speed)
        
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        sess.run(train, feed_dict={X:batch_xs, Y:batch_ys, keep_prob:0.7, lr:learning_rate})
        avg_cost += c/total_batch
    a, c = sess.run([accuracy, cost], feed_dict={X:mnist.test.images, Y:mnist.test.labels, keep_prob:1, lr:learning_rate})
    print(a,c, avg_cost/(e+1), learning_rate)
    
writer = tf.summary.FileWriter('sample2',session.graph)

In [36]:
xdata = [[0,0], [0,1], [1,0], [1,1]]
ydata = [[0], [1], [1], [0]]

X = tf.placeholder(tf.float32, shape=[None, 2])
Y = tf.placeholder(tf.float32, shape=[None, 1])

W1 = tf.Variable(tf.random_normal([2,2], stddev=0.1))
b1 = tf.Variable(tf.zeros([2]))
L1 = tf.nn.sigmoid(tf.matmul(X, W1)+b1)

W2 = tf.Variable(tf.random_normal([2,1]))
b2 = tf.Variable(tf.zeros([1]))
logits = tf.matmul(L1, W2)+b2
H = tf.nn.sigmoid(logits)
H2 = tf.round(H)

cost = tf.losses.mean_squared_error(H, Y)
train = tf.train.AdamOptimizer(0.05).minimize(cost)

is_correct = tf.equal(H2, Y)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

sess = tf.Session()
sess.run(tf.global_variables_initializer())

for i in range(10001):
    sess.run(train, feed_dict={X:xdata, Y:ydata})
    if i%1000 == 0:
        a, c = sess.run([accuracy, cost], feed_dict={X:xdata, Y:ydata})
        print(a, c)
y = sess.run(H, feed_dict={X:xdata})
for i in range(4):
    print(xdata[i], y[i])

0.5 0.28655723
1.0 0.0002927937
1.0 7.9128906e-05
1.0 3.4050317e-05
1.0 1.7268972e-05
1.0 9.466743e-06
1.0 5.4092657e-06
1.0 3.164304e-06
1.0 1.8767023e-06
1.0 1.1224008e-06
1.0 6.7458024e-07
[0, 0] [0.00071413]
[0, 1] [0.99926037]
[1, 0] [0.99926025]
[1, 1] [0.00104597]
